In [1]:
import sys
sys.path.append("../../..")
%load_ext autoreload
%autoreload 1
import lightning.pytorch as pl
import numpy as np

from ntupleReaders.clue_ntuple_reader import ClueNtupleReader

from ml.dynamic_reduction_network import DynamicReductionNetwork
%aimport ml.regression.drn.modules
%aimport ml.regression.drn.plot
%aimport ml.regression.drn.dataset_making
%aimport energy_resolution.sigma_over_e
from ml.regression.drn.modules import *
from ml.regression.drn.dataset_making import *
%aimport ml.regression.drn.callbacks
from ml.regression.drn.callbacks import *

2023-06-14 12:16:42.691290: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-14 12:16:42.921886: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/grid_mnt/vol_home/llr/cms/cuisset/mambaforge/envs/clustering-analysis-gpu2/lib/python3.10/site-packages/zfit/__init__.py:63: UserWarning: TensorFlow warnings are by default suppressed by zfit. In order to show them, set the environment variable ZFIT_DISABLE_TF_WARNINGS=0. In order to suppress the TensorFlow warnings AND thi

In [5]:
reader = ClueNtupleReader("v40", "cmssw", "sim_proton_v46_patchMIP")
datamodule = DRNDataModule(reader, datasetComputationClass=LayerClustersTensorMaker, transformFct=ratioTransform, multiprocess_loader=False)

class MockDRN(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.param = nn.Parameter(torch.tensor([1.]))
    def forward(self, x):
        return torch.normal(mean=torch.ones(x.num_graphs, device=x.x.device), std=torch.ones(x.num_graphs, device=x.x.device)*0.1)  * self.param

model = DRNModule(drn=MockDRN(), scheduler="default", loss="mse_ratio")
devices = [3]

In [6]:
callback = SigmaOverECallback(prediction_type="ratio", overlaySigmaOverEResults=[reader.loadSigmaOverEResults("rechits")], 
    every_n_epochs=1, multiprocess_fit="forkserver", fit_data="test", debug_mode=True)
#callback.on_validation_epoch_end(trainer, model)

In [7]:
trainer = pl.Trainer(accelerator="gpu", devices=devices, enable_progress_bar=True, fast_dev_run=False, logger=True, detect_anomaly=False, 
                     callbacks=[callback], limit_train_batches=0.05, limit_val_batches=0.05, max_epochs=1)
trainer.fit(model=model, datamodule=datamodule)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name | Type    | Params
---------------------------------
0 | drn  | MockDRN | 1     
---------------------------------
1         Trainable params
0         Non-trainable params
1         Total params
0.000     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/grid_mnt/vol_home/llr/cms/cuisset/mambaforge/envs/clustering-analysis-gpu2/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 96 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/grid_mnt/vol_home/llr/cms/cuisset/mambaforge/envs/clustering-analysis-gpu2/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 96 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/grid_mnt/vol_home/llr/cms/cuisset/mambaforge/envs/clustering-analysis-gpu2/lib/python3.10/s

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Start predicting!
Start fitting!


/grid_mnt/vol_home/llr/cms/cuisset/mambaforge/envs/clustering-analysis-gpu2/lib/python3.10/site-packages/zfit/__init__.py:63: UserWarning: TensorFlow warnings are by default suppressed by zfit. In order to show them, set the environment variable ZFIT_DISABLE_TF_WARNINGS=0. In order to suppress the TensorFlow warnings AND this warning, set ZFIT_DISABLE_TF_WARNINGS=1.
  warnings.warn(
/grid_mnt/vol_home/llr/cms/cuisset/mambaforge/envs/clustering-analysis-gpu2/lib/python3.10/site-packages/zfit/__init__.py:63: UserWarning: TensorFlow warnings are by default suppressed by zfit. In order to show them, set the environment variable ZFIT_DISABLE_TF_WARNINGS=0. In order to suppress the TensorFlow warnings AND this warning, set ZFIT_DISABLE_TF_WARNINGS=1.
  warnings.warn(
/grid_mnt/vol_home/llr/cms/cuisset/mambaforge/envs/clustering-analysis-gpu2/lib/python3.10/site-packages/zfit/__init__.py:63: UserWarning: TensorFlow warnings are by default suppressed by zfit. In order to show them, set the env

In [30]:
list(model.parameters())

[Parameter containing:
 tensor([0.9559], requires_grad=True)]